In [1]:
import math
import random
import matplotlib.pyplot as plt

# Question1: (3 pts) Difference between TD and MC

Read the attached TD-MC chapter, and the example 6.1 Driving to home. Can you imagine a scenario in which a TD update would be better on average than an Monte Carlo update? Give an example scenario—a description of past experience and a current state—in which you would expect the TD update to be better.


**According to my previous experience, I know the accurate time of driving home from the entrance of highway. Therefore, I can re-estimate my time once I enter the highway (TD methods). However, if you use MC methods, you can only update your time when you reach your home.  TD updates are better and it’s initially, cause it only depends on previous steps, while MD must wait until the end of an episode.**

In [2]:
actions=["up", "down", "left", "right"]

x_scope=5
y_scope=4

epsilon_start=0.4
epsilon_final=0.01

#The discount factor γ = 0.9.
discount_factor=0.9

learning_rate=0.1

q_value=dict()

episodes=3000

In [3]:
def calc_epsilon(t, epsilon_start=epsilon_start,
        epsilon_final=epsilon_final, epsilon_decay=episodes):
    if t<2000:
        epsilon = epsilon_final + (epsilon_start - epsilon_final) \
            * math.exp(-1. * t / epsilon_decay)
    else:
        epsilon=0.0
    return epsilon


def q_value_init():
    q_value.clear()
    for i in range(x_scope):
        for j in range(y_scope):
            state=(i, j)
            for action in actions:
                q_value[(state, action)]=0 


def step(state, action):
    next_x, next_y=state
    if action=="up":
        next_y += 1
    elif action=="down":
        next_y -= 1
    elif action=="left":
        next_x -= 1
    else:
        next_x += 1

    next_state=(next_x, next_y) 
    reward=0
    if next_x<0 or next_x>(x_scope-1) or next_y<0 or next_y>(y_scope-1):
        next_state=state
        reward=-1
    if (next_x== 3 and next_y==2):
        reward=-5
    if (next_x== 0 and next_y==3):
        reward=10
    
    return next_state, reward

#arg_max
def arg_max(state):
    q_value_list=[]
    for action in actions:
        q_value_list.append((q_value[(state, action)], action)) 
    random.shuffle(q_value_list)

    action=max(q_value_list)[-1]
    return action 


#e-greedy
def e_greedy(state):
    q_value_list=[]
    for action in actions:
        q_value_list.append((q_value[(state, action)], action)) 
    random.shuffle(q_value_list)

    if random.random()>epsilon:
        action=max(q_value_list)[-1]
    else:
        action=random.choice(q_value_list)[-1]
    return action

# Use off policy Q learning to learn the optimal values of Q* (s, a). Please submit your own code on calculating the Q*(s,a).


In [4]:
def q_learning(state):
    
    action=e_greedy(state)
    next_state, reward=step(state, action)   

    
    next_action=arg_max(next_state)
 
    
    estimate=reward+discount_factor*q_value[(next_state, next_action)]
    ##Assign new q_value

    q_value[(state, action)] = (1-learning_rate)*q_value[(state, action)] + learning_rate*estimate
    
    done = False
    ###Terminal State
    if state == (0, 3):
        done = True
    return next_state, reward, done

In [5]:
reward_list_2=[]
q_value_init()
for episode in range(episodes):
    ####Initail state
    state=(4, 0)
    while True:
        reward_sum=0
        epsilon=calc_epsilon(episode)
        next_state, reward,done=q_learning(state)
        reward_sum+=reward
        state = next_state
        if done:
            break
    reward_list_2.append(reward_sum)


# --1  (1pts) What is the Q*(s,a) for each pair of s and a?

In [6]:
print('Q*(s,a)')
act = 'state    '
for action in actions:
    act = act + action + '    '
print(act)
string=''           
for i in range(x_scope):
    for j in range(y_scope):
        string = "("+str(i)+", "+str(j)+")   "
        for action in actions:
            string = string + str(format(q_value[((i, j), action)],'.3f'))+"   "
        print(string)

Q*(s,a)
state    up    down    left    right    
(0, 0)   0.000   0.000   0.000   0.000   
(0, 1)   6.676   0.000   0.000   0.000   
(0, 2)   46.058   0.001   -0.093   5.003   
(0, 3)   41.631   39.818   41.631   47.368   
(1, 0)   1.802   0.000   0.000   7.172   
(1, 1)   32.186   0.003   0.276   3.025   
(1, 2)   47.365   19.016   27.244   38.352   
(1, 3)   46.368   42.609   52.632   42.632   
(2, 0)   0.001   0.000   1.430   20.709   
(2, 1)   37.572   7.643   3.986   10.791   
(2, 2)   42.632   29.734   37.425   28.379   
(2, 3)   41.632   38.368   47.368   38.368   
(3, 0)   21.619   20.007   12.256   25.174   
(3, 1)   25.752   20.527   26.188   27.971   
(3, 2)   38.368   24.412   37.092   29.947   
(3, 3)   37.368   29.532   42.632   34.532   
(4, 0)   27.971   24.174   22.656   24.173   
(4, 1)   31.078   25.174   25.173   26.971   
(4, 2)   34.532   27.971   29.532   30.078   
(4, 3)   33.532   31.078   38.368   33.532   


# --2  (1pts) What is the V*(s) for each s?

In [7]:
print('V*(s)')
for j in range(y_scope):
    v_value = ''
    for i in range(x_scope):
        row = i
        col = y_scope-j-1
        state=(row, col)
        q_value_list=[]
        for action in actions:
            q_value_list.append((q_value[(state, action)])) 
        max_qvalue  = max(q_value_list)
        if(row==0 and col==3):
            max_qvalue+=10
        elif(row==3 and col==2):
            max_qvalue-=5
        v_value = v_value+str(format(max_qvalue,'.3f'))+ '   '
    print(v_value)


V*(s)
57.368   52.632   47.368   42.632   38.368   
46.058   47.365   42.632   33.368   34.532   
6.676   32.186   37.572   27.971   31.078   
0.000   7.172   20.709   25.174   27.971   


# --3  (1pts) What are the actions of optimal policy?

**The optimal action is (4,0)->(4,1)->(4,2)->(4,3)->(3,3)->(2,3)->(1,3)->(0,3)**

# Use Sarsa algorithm to learn the optimal values of Q* (s, a). Please submit your own code on calculating the Q*(s,a).

In [8]:
def sarsa(state):
    action=e_greedy(state)
    next_state, reward=step(state, action)   
    
    next_action=e_greedy(next_state)

    estimate=reward+discount_factor*q_value[(next_state, next_action)]
    
    #assign new q_value
    q_value[(state, action)] = (1-learning_rate)*q_value[(state, action)] + learning_rate*estimate
    done = False
    ##Terminal state
    if state == (0, 3):
        done = True
    return next_state, reward, done 

In [9]:
if __name__=="__main__":
    reward_list_1=[]
    q_value_init()
    
    for episode in range(episodes):
        ####Initial State
        state=(4, 0)
        while True:
            reward_sum=0
            epsilon=calc_epsilon(episode)
            next_state, reward,done=sarsa(state)
            reward_sum+=reward
            state = next_state
            if done:
                break
        reward_list_1.append(reward_sum)


# --1  (1pts) What is the Q*(s,a) for each pair of s and a?

In [10]:
print('Q*(s,a)')
act = 'state    '
for action in actions:
    act = act + action + '    '
print(act)
string=''           
for i in range(x_scope):
    for j in range(y_scope):
        string = "("+str(i)+", "+str(j)+")   "
        for action in actions:
            string = string + str(format(q_value[((i, j), action)],'.3f'))+"   "
        print(string)

Q*(s,a)
state    up    down    left    right    
(0, 0)   30.305   14.368   18.381   13.571   
(0, 1)   35.079   24.146   27.140   23.062   
(0, 2)   52.631   29.451   33.569   30.274   
(0, 3)   28.715   47.368   28.975   31.747   
(1, 0)   38.368   21.734   22.584   20.015   
(1, 1)   42.631   22.888   27.446   21.290   
(1, 2)   31.002   26.298   47.368   23.320   
(1, 3)   24.253   19.017   40.024   12.317   
(2, 0)   22.006   18.681   34.531   16.779   
(2, 1)   19.698   17.264   25.425   14.614   
(2, 2)   11.942   15.194   29.521   1.626   
(2, 3)   2.832   7.785   26.818   0.686   
(3, 0)   16.469   15.741   31.078   14.525   
(3, 1)   4.349   11.605   23.020   5.687   
(3, 2)   1.214   4.231   18.130   0.189   
(3, 3)   -0.772   -1.342   8.404   -0.026   
(4, 0)   12.112   14.180   27.970   12.987   
(4, 1)   0.796   14.903   6.291   4.718   
(4, 2)   0.005   4.867   -1.355   -0.553   
(4, 3)   -0.410   -0.071   0.426   -0.568   


# --2  (1pts) What is the V*(s) for each s?

In [11]:
print('V*(s)')
for j in range(y_scope):
    v_value = ''
    for i in range(x_scope):
        row = i
        col = y_scope-j-1
        state=(row, col)
        q_value_list=[]
        for action in actions:
            q_value_list.append((q_value[(state, action)])) 
        max_qvalue = max(q_value_list)
        if(row==0 and col==3):
            max_qvalue+=10
        elif(row==3 and col==2):
            max_qvalue-=5
        v_value = v_value+str(format(max_qvalue,'.3f'))+ '   '
    print(v_value)


V*(s)
57.368   40.024   26.818   8.404   0.426   
52.631   47.368   29.521   13.130   4.867   
35.079   42.631   25.425   23.020   14.903   
30.305   38.368   34.531   31.078   27.970   


# --3  (1pts) What are the actions of optimal policy?

**The optimal action is (4,0)->(3,0)->(2,0)->(1,0)->(1,1)->(1,2)->(0,2)->(0,3)**

# Please discuss your observed difference between Sarsa algorithm and  off-policy Q-learning (1pts) in your game.

Q-learning learns values for the optimal policy which close to the penality, while Saras far away from the penality. 
Although Q- learning actually learns the values of the optimal policy, its on-line performance is worse than that of Sarsa, which learns the roundabout policy.
Of course, if e were gradually reduced, then both methods would asymptotically converge to the same policy.